# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Context-sensitive spelling correction

In [ ]:
import re
import numpy as np
import random
# I used random for adding errors for test cases
# but fixing seed did not help)
random.seed(26)
np.random.seed(26)

In [300]:
# extract all words from the corpus
def process_corpus(corpus_filename):
    with open(corpus_filename) as f:
        corpus = f.read()
        # find words
        lowercased_corpus = corpus.lower()
        all_words = re.findall(r'\w+', lowercased_corpus)
        for w in all_words:
            if w == 'I':
                print('Found')
                break
        unique_words = set(all_words)
    return all_words, unique_words

# construct dictionary of word frequencies
def get_words_frequencies(all_words):
    word_freq_dict = {}
    for word in all_words:
        if word in word_freq_dict:
            word_freq_dict[word] += 1
        else:
            word_freq_dict[word] = 1
    return word_freq_dict

# get the probability of a word
def get_word_prob(word, all_words, word_freq_dict):
    # check that the word exists in the vocabulary
    if word in word_freq_dict:
        return word_freq_dict[word] / len(all_words)
    
    return 0

# function to generate words from the given one by adding 1 character
def add_char(word):
    words_with_char_added = []
    possible_chars = 'qwertyuiopasdfghjklzxcvbnm'
    for i in range(len(word)):
        for char in possible_chars:
            words_with_char_added.append(word[:i] + char + word[i:])
        words_with_char_added.append(word + char)
    return words_with_char_added

# function to generate words from the given one by deleting 1 character
def delete_char(word):
    words_with_char_deleted = []
    for i in range(len(word)):
        words_with_char_deleted.append(word[:i] + word[i+1:])
    return words_with_char_deleted

# function to generate words from the given one by replacing 1 character
def replace_char(word):
    words_with_char_replaced = []
    possible_chars = 'qwertyuiopasdfghjklzxcvbnm'
    for i in range(len(word)):
        for char in possible_chars:
            new_word = word[:i] + char + word[i+1:]
            words_with_char_replaced.append(new_word)
    return words_with_char_replaced

# function to generate words from the given one by swapping 2 characters
def swap_chars(word):
    words_with_chars_swapped = []
    for i in range(len(word)-1):
        new_word = word[:i] + word[i+1] + word[i] + word[i+2:]
        words_with_chars_swapped.append(new_word)
    return words_with_chars_swapped

# filter only words that are present in the vocabulary
def filter_existent_words(words, vocabulary):
    return [word for word in words if word in vocabulary]

# check the word presence in the vocabulary
def check_existence(word, vocabulary):
    if word in vocabulary:
        return True
    return False

# function for generating words obtained by adding, deleting, 
# replacing or swapping 1 character
def generate_candidates_edit_1(word):
    candidates = []
    words_with_char_added = add_char(word)
    words_with_char_deleted = delete_char(word)
    words_with_char_replaced = replace_char(word)
    words_with_chars_swapped = swap_chars(word)

    candidates.extend(words_with_char_added)
    candidates.extend(words_with_char_deleted)
    candidates.extend(words_with_char_replaced)
    candidates.extend(words_with_chars_swapped)
    unique_candidate_words = set(candidates)
    
    return unique_candidate_words


## Norwig's solution (Unigram model)

The main idea of this model is to generate words that are similar to the given one and choose the best one based on the frequency of it in the corpus.

This model was chosen for the first implementaton because of its simplicity. Nevertheless, it does not capture the context and its usage is reasonable for correction of single words.

As the example of the model implementation I used the [Norwig's solution](https://norvig.com/spell-correct.html) and [video tutorial](https://www.youtube.com/watch?v=4yOKlWZk52M).

To compare my model implementation with the official Norwig's model as a corpus I decided to use `big.txt` dataset.


In [301]:
all_words, unique_words = process_corpus('big.txt')
print(f"Num of words in corpus: {len(all_words)}")
print(f"Num of unique words in corpus: {len(unique_words)}")

Num of words in corpus: 1115585
Num of unique words in corpus: 32198


In [302]:
word_freq = get_words_frequencies(all_words)
word_freq['cat']

10

In [303]:
# function to correct single word
def correct_word_simple(word, vocabulary):
    word = word.lower()
    if check_existence(word, vocabulary):
        return word
    unique_candidates_edit_1 = generate_candidates_edit_1(word)
    candidates_edit_2 = []
    for candidate in unique_candidates_edit_1:
        new_cadidates_edit_2 = generate_candidates_edit_1(candidate)
        candidates_edit_2.extend(new_cadidates_edit_2)
    
    unique_candidates_edit_2 = set(candidates_edit_2)

    all_candidates = []
    unique_candidates_edit_1_existent = filter_existent_words(unique_candidates_edit_1, vocabulary)
    unique_candidates_edit_2_existent = filter_existent_words(unique_candidates_edit_2, vocabulary)
    for candidate in unique_candidates_edit_1_existent:
        all_candidates.append((candidate, 1))
    for candidate in unique_candidates_edit_2_existent:
        all_candidates.append((candidate, 2))
    unique_candidates = set(all_candidates)

    # sort unique_candidates by the distance and the probability of the word
    sorted_candidates = sorted(unique_candidates, key=lambda x: (x[1], -get_word_prob(x[0], all_words, word_freq)))
    if len(sorted_candidates) > 0:
        best_candidate = sorted_candidates[0]
    else:
        best_candidate = (word, 0)

    return best_candidate[0]

# function to correct the text using Norwig's idea
def correct_text(given_text):
    found_words = re.finditer(r'\b\w+\b', given_text)
    cur_idx = 0
    corrected_text = []
    for cur_word_with_boundaries in found_words:
        word = cur_word_with_boundaries.group()
        start_idx, end_idx = cur_word_with_boundaries.span()
        corrected_word = correct_word_simple(word, unique_words)
        # to save the spaces and punctuation
        corrected_text.append(given_text[cur_idx:start_idx])
        # if the word's characters are all UPPER
        if word.isupper():
            corrected_word = corrected_word.upper()
        # if the first letter is in upper case
        elif word.istitle():
            corrected_word = corrected_word.capitalize()
    
        corrected_text.append(corrected_word)
        cur_idx = end_idx
    corrected_text.append(given_text[cur_idx:])
    corrected_text_result = ''.join(corrected_text)
        
    return corrected_text_result


### Examples of the model outputs

In [304]:
correct_word_simple('speling', unique_words)

'spelling'

In [305]:
corrected_text = correct_text("I am a student7!")
print(corrected_text)

I am a student!


In [306]:
text = 'It is speling correction task.'
corrected_text = correct_text(text)
print(corrected_text)

It is spelling correction task.


In [307]:
text_example = 'dking sport'
corrected_example = correct_text(text_example)
print(corrected_example)

king sport


In [308]:
text_example2 = 'dking species'
corrected_example2 = correct_text(text_example2)
print(corrected_example2)

king species


In [309]:
prob_king = get_word_prob('king', all_words, word_freq)
prob_doing = get_word_prob('doing', all_words, word_freq)
prob_dying = get_word_prob('dying', all_words, word_freq)
print(f"Probability of the word 'king': {prob_king}, frequency: {word_freq['king']}")
print(f"Probability of the word 'doing': {prob_doing}, frequency: {word_freq['doing']}")
print(f"Probability of the word 'dying': {prob_dying}, frequency: {word_freq['dying']}")

Probability of the word 'king': 0.00021334098253382753, frequency: 238
Probability of the word 'doing': 0.00015955754155891303, frequency: 178
Probability of the word 'dying': 7.171125463321934e-05, frequency: 80


From the above cell output we can see that since the word `king`occurs more often in the training corpus, then the model tends to choose it instead of words `doind` and `dying` that are more suitable in this case.

## Bigram model

The main idea of the bigram model is to choose the most appropriate word not just by the frequency of its occurance in the corpus but by the frequency of occurance of the current word together with the previous one. This distinction giveas the opportunity to understand the context and avoid the problem of choosing the most frequent word from the generated candidates.

As a training corpus I decided to choose the datasets `count_1w.txt` (for single word frequencies) and `count_2w.txt` (for bigram frequencies). I found them in the [website](http://norvig.com/ngrams/).

Firstly, I implemented simple probability calculation. After that I decided to add Laplase smoothing to avoid 0 probability for unseen bigrams as was stated in the [article](https://web.stanford.edu/~jurafsky/slp3/3.pdf) about N-gram language models.

In [ ]:
# get single word frequencies
def process_1_word_freq(filename):
    with open(filename) as f:
        word_freq = {}
        for line in f:
            word, freq = line.strip().split()
            word = word.lower()
            word_freq[word] = int(freq)
    return word_freq

# get bigrams frequencies
def process_2_word_freq(filename):
    with open(filename) as f:
        word_freq = {}
        for line in f:
            word1, word2, freq = line.strip().split()
            word1 = word1.lower()
            word2 = word2.lower()
            bigram = word1 + ' ' + word2
            word_freq[bigram] = int(freq)
    return word_freq

# without Laplase smoothing
def calculate_bigram_prob(prev_word, cur_word, bigram_freq, single_word_freq):
    lowered_prev_word = prev_word.lower()
    lowered_cur_word = cur_word.lower()
    bigram = lowered_prev_word + ' ' + lowered_cur_word
    total_single_word_freq = sum(single_word_freq.values())
    if bigram in bigram_freq:
        if lowered_prev_word in single_word_freq:
            return bigram_freq[bigram] / single_word_freq[lowered_prev_word]
        else:
            return bigram_freq[bigram] / total_single_word_freq
    else:
        if lowered_cur_word in single_word_freq:
            return single_word_freq[lowered_cur_word] / total_single_word_freq
        else:
            return 0
        
# adding Laplase smoothing
def calculate_bigram_prob_with_laplase_smoothing(prev_word, cur_word, bigram_freq, single_word_freq):
    lowered_prev_word = prev_word.lower()
    lowered_cur_word = cur_word.lower()
    bigram = lowered_prev_word + ' ' + lowered_cur_word
    bigram_count = bigram_freq.get(bigram, 0)
    prev_word_count = single_word_freq.get(lowered_prev_word, 0)
    smoothed_prob = (bigram_count + 1)/ (prev_word_count + len(unique_words))
    return smoothed_prob

# function to calculate the probability of the word sequence based on the bigram probabilities
def calculate_word_sequence_prob(words, bigram_freq, single_word_freq, prev_token = '<S>', edit_distance=1):
    result = 0
    for i in range(len(words)):
        if i==0:
            prob = calculate_bigram_prob(prev_token, words[i], bigram_freq, single_word_freq)
        else:
            prob = calculate_bigram_prob(words[i-1], words[i], bigram_freq, single_word_freq)
        if prob == 0:
            prob = 1e-10
        result+= np.log(prob)
        
        # penalize for number of corrections
        result = result - 0.05*edit_distance
        # print(result)
    return result

def calculate_word_sequence_prob_with_laplase_smoothing(words, bigram_freq, single_word_freq, prev_token = '<S>', edit_distance=1):
    result = 0
    for i in range(len(words)):
        if i==0:
            prob = calculate_bigram_prob_with_laplase_smoothing(prev_token, words[i], bigram_freq, single_word_freq)
        else:
            prob = calculate_bigram_prob_with_laplase_smoothing(words[i-1], words[i], bigram_freq, single_word_freq)
        if prob == 0:
            prob = 1e-10
        result+= np.log(prob)
        
        # penalize for number of corrections
        result = result - 0.05*edit_distance
        # print(result)
    return result


In [311]:
single_word_freq = process_1_word_freq('count_1w.txt')
bigram_freq = process_2_word_freq('count_2w.txt')

In [312]:
prob = calculate_bigram_prob('the', 'cat', bigram_freq, single_word_freq)
print(prob)

7.259827996986489e-05


In [313]:
prob = calculate_bigram_prob('t', 'cot', bigram_freq, single_word_freq)
print(prob)

3.079682723190858e-06


In [314]:
word_seq_prob = calculate_word_sequence_prob(['the', 'cat'], bigram_freq, single_word_freq)
print(word_seq_prob)

-17.360431386064402


In [315]:
word_seq_prob = calculate_word_sequence_prob(['t', 'cot'], bigram_freq, single_word_freq)
print(word_seq_prob)

-22.226910528077223


In [ ]:
def correct_word_bigram(given_word, given_text_tokens, given_word_idx):
    given_word_lower = given_word.lower()
    
    # if the word is correct, then do not generate candidates
    if check_existence(given_word_lower, unique_words):
        return given_word

    # generate edit 1 distance candidates
    all_candidates_with_edit_dist = []
    unique_candidates_edit_1 = generate_candidates_edit_1(given_word_lower)
    unique_candidates_edit_1_existent = filter_existent_words(unique_candidates_edit_1, unique_words)

    for candidate in unique_candidates_edit_1_existent:
        all_candidates_with_edit_dist.append((candidate, 1))
    
    # generate edit 2 distance candidates
    candidates_edit_2 = []
    for candidate in unique_candidates_edit_1:
        new_candidates_edit_2 = generate_candidates_edit_1(candidate)
        candidates_edit_2.extend(new_candidates_edit_2)
    
    unique_candidates_edit_2_existent = filter_existent_words(set(candidates_edit_2), unique_words)
    for candidate in unique_candidates_edit_2_existent:
        all_candidates_with_edit_dist.append((candidate, 2))
    
    all_unique_candidates_with_edit_dist = set(all_candidates_with_edit_dist)

    # if there are no candidates that are present in the vocabulary, return the original word
    if not all_unique_candidates_with_edit_dist:
        return given_word 
    

    # for each candidate calculate the probability of the word sequence with this candidate
    # and select one with the larger probability
    new_probabilities = []
    for (candidate, edit_dist) in all_unique_candidates_with_edit_dist:
        new_word_sequence = given_text_tokens.copy()
        new_word_sequence[given_word_idx] = candidate
        prob = calculate_word_sequence_prob(new_word_sequence, bigram_freq, single_word_freq, edit_distance=edit_dist)
        new_probabilities.append(prob)
    best_candidate = list(all_unique_candidates_with_edit_dist)[new_probabilities.index(max(new_probabilities))][0]

    # save word in the given format
    if given_word.isupper():
        return best_candidate.upper()
    elif given_word.istitle():
        return best_candidate.capitalize()
    else:
        return best_candidate
    
def correct_word_bigram_with_laplase_smoothing(given_word, given_text_tokens, given_word_idx):
    given_word_lower = given_word.lower()
    
    # if the word is correct, then do not generate candidates
    if check_existence(given_word_lower, unique_words):
        return given_word

    # generate edit 1 distance candidates
    all_candidates_with_edit_dist = []
    unique_candidates_edit_1 = generate_candidates_edit_1(given_word_lower)
    unique_candidates_edit_1_existent = filter_existent_words(unique_candidates_edit_1, unique_words)

    for candidate in unique_candidates_edit_1_existent:
        all_candidates_with_edit_dist.append((candidate, 1))
    
    # generate edit 2 distance candidates
    candidates_edit_2 = []
    for candidate in unique_candidates_edit_1:
        new_candidates_edit_2 = generate_candidates_edit_1(candidate)
        candidates_edit_2.extend(new_candidates_edit_2)
    
    unique_candidates_edit_2_existent = filter_existent_words(set(candidates_edit_2), unique_words)
    for candidate in unique_candidates_edit_2_existent:
        all_candidates_with_edit_dist.append((candidate, 2))
    
    all_unique_candidates_with_edit_dist = set(all_candidates_with_edit_dist)

    # if there are no candidates that are present in the vocabulary, return the original word
    if not all_unique_candidates_with_edit_dist:
        return given_word 
    

    # for each candidate calculate the probability of the word sequence with this candidate
    # and select one with the larger probability
    new_probabilities = []
    for (candidate, edit_dist) in all_unique_candidates_with_edit_dist:
        new_word_sequence = given_text_tokens.copy()
        new_word_sequence[given_word_idx] = candidate
        prob = calculate_word_sequence_prob_with_laplase_smoothing(new_word_sequence, bigram_freq, single_word_freq, edit_distance=edit_dist)
        new_probabilities.append(prob)
    best_candidate = list(all_unique_candidates_with_edit_dist)[new_probabilities.index(max(new_probabilities))][0]

    # save word in the given format
    if given_word.isupper():
        return best_candidate.upper()
    elif given_word.istitle():
        return best_candidate.capitalize()
    else:
        return best_candidate

def correct_text_bigram(given_text):
    found_words = list(re.finditer(r'\b\w+\b', given_text))
    corrected_text = []
    cur_idx = 0

    for idx, match in enumerate(found_words):
        word = match.group()
        start, end = match.span()

        # save the text before the word because of spaces and punctuation to reconstruct the initial text
        corrected_text.append(given_text[cur_idx:start])
        corrected_word = correct_word_bigram(word, [m.group() for m in found_words], idx)
        corrected_text.append(corrected_word)
        cur_idx = end

    # save the text after the last word because of spaces and punctuation
    corrected_text.append(given_text[cur_idx:])
    return "".join(corrected_text)

def correct_text_bigram_with_laplase_smoothing(given_text):
    found_words = list(re.finditer(r'\b\w+\b', given_text))
    corrected_text = []
    cur_idx = 0

    for idx, match in enumerate(found_words):
        word = match.group()
        start, end = match.span()

        # save the text before the word because of spaces and punctuation to reconstruct the initial text
        corrected_text.append(given_text[cur_idx:start])
        corrected_word = correct_word_bigram_with_laplase_smoothing(word, [m.group() for m in found_words], idx)
        corrected_text.append(corrected_word)
        cur_idx = end

    # save the text after the last word because of spaces and punctuation
    corrected_text.append(given_text[cur_idx:])
    return "".join(corrected_text)


### Examples of bigram model outputs

In [318]:
correct_text_bigram('Hello! I am a student7.')

'Hello! I am a student.'

In [319]:
correct_text_bigram('dking sport')

'diving sport'

In [320]:
correct_text_bigram('dking species')

'diving species'

### Examples of bigram model with Laplase smoothing outputs

In [342]:
correct_text_bigram_with_laplase_smoothing('Hello! I am a student7.')

'Hello! I am a student.'

In [321]:
correct_text_bigram_with_laplase_smoothing('dking sport')

'ing sport'

In [322]:
correct_text_bigram_with_laplase_smoothing('dking species')

'ing species'

In [323]:
word='dking'
candidates = generate_candidates_edit_1(word)
print(f"Candidates for '{word}': {candidates}")

Candidates for 'dking': {'dkinyg', 'dving', 'doing', 'uking', 'dkaing', 'dkwing', 'dkidg', 'sking', 'daking', 'dkiny', 'dtking', 'dkinvg', 'dkifg', 'dkinz', 'dkitng', 'dkong', 'dkingg', 'hking', 'dkixng', 'djking', 'xking', 'dkivng', 'dkilng', 'dning', 'dkiung', 'dkinx', 'dling', 'dying', 'dkinc', 'dkiyng', 'dkeing', 'dkixg', 'dkijng', 'dkicg', 'bdking', 'dkinl', 'dkxing', 'dfing', 'dkibng', 'dkieg', 'dkinw', 'gdking', 'dkiqng', 'bking', 'dkipg', 'dkiang', 'dkcng', 'kking', 'mdking', 'dcking', 'dikng', 'diing', 'dkinig', 'fking', 'dkpng', 'dqking', 'wking', 'zking', 'dkina', 'dkind', 'dcing', 'dvking', 'dkinpg', 'edking', 'ldking', 'yking', 'dkinsg', 'dkigng', 'dkine', 'dkning', 'dfking', 'dgking', 'dkinjg', 'dkang', 'dkisng', 'dkinag', 'dkiag', 'dkyng', 'dwing', 'dging', 'dkfng', 'fdking', 'dkindg', 'rdking', 'dkinb', 'cdking', 'dkizg', 'dknng', 'idking', 'dkink', 'dkding', 'dkilg', 'xdking', 'dkting', 'dxking', 'dkqing', 'dkino', 'dkping', 'duking', 'dsing', 'dbing', 'dkinkg', 'dkrng

In [ ]:
print("Bigram count for doing sport:", bigram_freq.get("doing sport", 0))
print("Bigram count for dying species:", bigram_freq.get("dying species", 0))
print("Single word count for dying:", single_word_freq.get("dying", 0))
print("Single word count for doing:", single_word_freq.get("doing", 0))

Bigram count: 0
Bigram count: 0
Single word count for dying: 9123557
Single word count for doing: 80821946


In [343]:
print("Single word count for ing:", single_word_freq.get("ing", 0))

Single word count for ing: 17637606


In the datasets used for training language modela there were no bigrams for `doing sport` and no bigrams for `dying species`. Therefore, for the test case `dking sport` and `dking species` the model gives not expected results: `diving sport` and `diving species`.

## Decisions justification

### Datasets

**Training corpus**

I used the datasets from the [website](http://norvig.com/ngrams/) because this makes it possible to compare the results of Norvig's original model and my implementation, since the results of such models depend on the corpus on which they were trained.

In the article about N-gram models there was an idea about back-off: to backtrack to N-1 gram if there is no N-gram in the dataset and calculate the probability of this N-gram through a linear combination of probabilities from the unigram to the N-gram. I wanted to implement trigram model but I did not find the appropriate dataset. For this reason I tried to extract unigrams, bigrams and trigrams from the nltk corpuses (brown) and train the language model on it. However, this does not solve the problem with `dking sport` and `dking species` correction and I kept the models trained on datasets close to the official implementation to be able to compare. 

**Test datasets**

I tested the model on 5 test cases. 

2 of them (words from unit test and from the `spell-testset1.txt`) were mentioned in the official solution. I took them to compare the results. 

The other 3 includes:
- sentence generated from the bigram corpus `count_2w.txt` with randomly generated spelling mistakes (to test model in the conditions similarto training)
- paragraph generated from the bigram corpus `count_2w.txt` with randomly generated spelling mistakes (to test model in the conditions similarto training)
- text fragment from the book "The Lord of The Rings" by John Ronald Reuel Tolkien (difficult test case for the model because there are a lot of unseen words)

### Weights for edit1 and edit2 distances

I decided to penalize the candidates by reducing their probability by `0.05*edit_distance`. The number `0.05` was chosen by experiments and worked well. Edit1 word will be selected with higher probability than the edit2 candidate in case of similar probabilities.


### Absent word probabilities

To avoid problem with the similarly small probabilities for unseen words or bigrams I slightly modified the probability calculation for the simple bigram model by considering different cases. Moreover, I added Laplase smoothing that worked better.

### Metric for evaluation

To evaluate the models I have chosen corrected word accuracy metric. It is calculated as (the number of correctly corrected words)/(the number of words with errors that are present in the dataset). In my opinion this metric is honest because we cannot evaluate the model on the words that were not present in the corpus.


### Difficulties

1. **Probability calculation**

Multiplication of probabilities fastly becomes 0 => use sum of logarithms of probabilities instead of raw probability multiplication.

2. **Capturing context** - moving from unigrams to bigrams

With unigrams for phrases of 2 words the context is not captured. Therefore, I decided to use bigram models.

3. **Corpus**

- large size of the file with huge corpus

To get better results the model should process a large corpus.After trying to use a large dataset, my computer got sick and I decided to postpone this idea and think about other ways to improve the result. For example, calculating the probability of bigram in the case of its absence in the training corpus 😅

- no appropriate trigram dataset

Since I had no success in finding an open suitable dataset with trigrams, I tried to train the model on the brown corpus from nltk, but it did not help to improve the result significantly. Therefore, for the convenience of comparing my results with the official solution of Norvig, I decided to leave the datasets given on his website.

4. **Probabilities for unseen words**

The approach with the usual calculation of probabilities for both unigrams and bigrams is not very suitable, because for words or bigrams that the model sees for the first time the probability will be 0. Therefore, to calculate the probability in the usual bigram, I considered the possible options and for each of them suggested how the probability could be calculated to avoid zero probabilities. In addition, I tried to use Laplase smoothing, which showed good results, even though the authors of the paper highlighted that it was not the best idea, but it works.



## Evaluation on a test set

### Comparing my unigram model with Norwig's

I noticed slight difference in the training corpus because the size of the vocabulary in my case is 32198 and in Norwig's solution is 32192. I suppose that the file `big.txt` could have been changed a bit. But values like word count don't differ much, so I just removed the check for equality of such values to the Norvig corpus values from the unit tests.

In [ ]:
# Norvig has 32192
len(unique_words)

32198

In [ ]:
# Norvig has 1115504
len(all_words)

1115585

In [ ]:
# Norvig has 79808
word_freq['the']

79809

In [329]:
get_word_prob('quintessential', all_words, word_freq)

0

In [330]:
get_word_prob('the', all_words, word_freq)

0.07154004401278254

### Norvig tests

In [ ]:
# Norvig tests
def unit_tests():
    assert correct_text('speling') == 'spelling'              # insert
    assert correct_text('korrectud') == 'corrected'           # replace 2
    assert correct_text('bycycle') == 'bicycle'               # replace
    assert correct_text('inconvient') == 'inconvenient'       # insert 2
    assert correct_text('arrainged') == 'arranged'            # delete
    assert correct_text('peotry') =='poetry'                  # transpose
    assert correct_text('peotryy') =='poetry'                 # transpose + delete
    assert correct_text('word') == 'word'                     # known
    assert correct_text('quintessential') == 'quintessential' # unknown
    assert get_word_prob('quintessential', all_words, word_freq) == 0
    assert 0.07 < get_word_prob('the', all_words, word_freq) < 0.08
    return 'unit_tests pass'

### For unigram model

In [332]:
def spelltest(tests, verbose=True):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.time()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correct_word_simple(wrong, unique_words)
        good += (w == right)
        if w != right:
            unknown += (right not in unique_words)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, word_freq.get(w, 0), right, word_freq.get(right, 0)))
    dt = time.time() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))
    
def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

print(unit_tests())
spelltest(Testset(open('spell-testset1.txt')))

unit_tests pass
correction(contende) => contend (3); expected contented (13)
correction(contended) => contended (9); expected contented (13)
correction(proplen) => people (891); expected problem (71)
correction(guic) => guns (111); expected juice (5)
correction(juce) => june (44); expected juice (5)
correction(jucie) => julie (71); expected juice (5)
correction(juise) => guise (8); expected juice (5)
correction(juse) => just (767); expected juice (5)
correction(localy) => local (181); expected locally (10)
correction(compair) => company (190); expected compare (29)
correction(transportibility) => transportibility (0); expected transportability (0)
correction(miniscule) => miniscule (0); expected minuscule (0)
correction(poartry) => party (298); expected poetry (10)
correction(stanerdizing) => stanerdizing (0); expected standardizing (0)
correction(futher) => father (533); expected further (138)
correction(biscutes) => disputes (27); expected biscuits (8)
correction(receit) => recent (5

### For bigram model

In [333]:
def spelltest(tests, verbose=True):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.time()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correct_text_bigram(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in unique_words)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, word_freq.get(w, 0), right, word_freq.get(right, 0)))
    dt = time.time() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))
    
def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

print(unit_tests())
spelltest(Testset(open('spell-testset1.txt')))

unit_tests pass
correction(contenpted) => contested (4); expected contented (13)
correction(contende) => content (29); expected contented (13)
correction(contended) => contended (9); expected contented (13)
correction(contentid) => content (29); expected contented (13)
correction(begining) => refining (1); expected beginning (143)
correction(problam) => program (43); expected problem (71)
correction(proble) => noble (48); expected problem (71)
correction(proplen) => people (891); expected problem (71)
correction(dirven) => dive (1); expected driven (66)
correction(guic) => gulf (13); expected juice (5)
correction(juce) => suck (3); expected juice (5)
correction(juise) => jose (1); expected juice (5)
correction(juse) => jose (1); expected juice (5)
correction(localy) => lonely (13); expected locally (10)
correction(compair) => complain (14); expected compare (29)
correction(transportibility) => transportibility (0); expected transportability (0)
correction(miniscule) => miniscule (0); e

### For bigram model with Laplase smoothing

In [334]:
def spelltest(tests, verbose=True):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.time()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correct_text_bigram_with_laplase_smoothing(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in unique_words)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, word_freq.get(w, 0), right, word_freq.get(right, 0)))
    dt = time.time() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))
    
def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

print(unit_tests())
spelltest(Testset(open('spell-testset1.txt')))

unit_tests pass
correction(contende) => content (29); expected contented (13)
correction(contended) => contended (9); expected contented (13)
correction(contentid) => content (29); expected contented (13)
correction(problam) => program (43); expected problem (71)
correction(proble) => people (891); expected problem (71)
correction(proplen) => people (891); expected problem (71)
correction(dirven) => given (364); expected driven (66)
correction(exstacy) => eustace (1); expected ecstasy (8)
correction(guic) => music (56); expected juice (5)
correction(juce) => such (1436); expected juice (5)
correction(jucie) => judge (45); expected juice (5)
correction(juise) => use (320); expected juice (5)
correction(juse) => else (201); expected juice (5)
correction(localy) => local (181); expected locally (10)
correction(transportibility) => transportibility (0); expected transportability (0)
correction(miniscule) => miniscule (0); expected minuscule (0)
correction(aranged) => range (39); expected a

In [ ]:
def add_errors(correct_text, error_rate = 0.2):
    found_words = list(re.finditer(r'\b\w+\b', correct_text))
    corrupted_text = []
    cur_idx = 0

    num_of_errors = int(len(found_words) * error_rate)
    error_indices = random.sample(range(len(found_words)), num_of_errors)

    for idx, match in enumerate(found_words):
        word = match.group()
        start, end = match.span()

        # append the text before the word ro save spaces and punctuation
        corrupted_text.append(correct_text[cur_idx:start])

        # randomly add errors
        if idx in error_indices and len(word) > 1:
            error_type = random.choice(["add", "delete", "replace", "swap"])
            if error_type == "add":
                corrupted_word = random.choice(add_char(word))
            elif error_type == "delete":
                corrupted_word = random.choice(delete_char(word)) if len(word) > 2 else word
            elif error_type == "replace":
                corrupted_word = random.choice(replace_char(word))
            elif error_type == "swap":
                corrupted_word = random.choice(swap_chars(word))
        else:
            corrupted_word = word 

        corrupted_text.append(corrupted_word)

        cur_idx = end

    # save the remaining puntuation and spaces that are not words
    corrupted_text.append(correct_text[cur_idx:])

    return "".join(corrupted_text), num_of_errors

# function to calculate the model accuracy as the (number of correctly corrected words) / (number of all words with errors that are present in the corpus)
def calculate_word_accuracy(original, corrupted, corrected):
    initial_words = re.findall(r'\b\w+\b', original)
    words_with_errors = re.findall(r'\b\w+\b', corrupted)
    corrected_words = re.findall(r'\b\w+\b', corrected)
    correctly_corrected_words_count = 0
    possible_corrected_words = 0

    for initial_word, word_with_error, corrected_word in zip(initial_words, words_with_errors, corrected_words):
        if initial_word in unique_words:
            possible_corrected_words+=1
            if initial_word == corrected_word and initial_word != word_with_error:
                correctly_corrected_words_count+=1
    
    if possible_corrected_words == 0:
        return 1
    else:
        return correctly_corrected_words_count / possible_corrected_words

### Test sentence 

In [337]:
test_sentence_1 = "The autumn season brought colorful leaves as artists prepared for an annual exhibition showcasing contemporary artwork."
test_sentence_1_with_errors = "The hutumn season brought colorful leaves as artists prepared for an sannual exhibition showcasing conetmporary artwork."

corrected_unigram = correct_text(test_sentence_1_with_errors)
corrected_bigram = correct_text_bigram(test_sentence_1_with_errors)
corrected_bigram_laplase_smoothing = correct_text_bigram_with_laplase_smoothing(test_sentence_1_with_errors)

print("Unigram accuracy:", calculate_word_accuracy(test_sentence_1, test_sentence_1_with_errors, corrected_unigram))
print("Bigram accuracy:", calculate_word_accuracy(test_sentence_1, test_sentence_1_with_errors, corrected_bigram))
print("Bigram with Laplase smoothing accuracy:", calculate_word_accuracy(test_sentence_1, test_sentence_1_with_errors, corrected_bigram_laplase_smoothing))

Unigram accuracy: 0.25
Bigram accuracy: 0.25
Bigram with Laplase smoothing accuracy: 0.25


### Test paragraph

In [338]:
test_paragraph = """As the sun set, an art exhibition opened in the city's cultural center, showcasing contemporary artwork from renowned 
and emerging visual artists. The gallery was filled with vibrant paintings, abstract sculptures, and multimedia installations that explored 
themes of identity and transformation. Art enthusiasts and collectors engaged in thoughtful discussions about the impact of modern art on society. 
Meanwhile, the event organizers prepared for an evening panel featuring well-known creative professionals discussing the future of digital media 
in the artistic landscape."""

# added some mistakes using add_errors function. To reproduce results I fixed the paragraph.
test_paragraph_with_errors = """As the sun set, an art exhisbition opened in the ity's cultural cehnter, showcasing contemporary artwork from renowned 
and emergin visual artists. The gallery was flilled with vibrakt paintings, abstract sculptures, and multimedia installations taht explored 
themes fo identity and transformation. Art enthusiasts and collectors engaged in thoughtful discussions about the impact of modern art fn societjy. 
Meanwhile, th event orgainzers prepared ofr an evening panel featuring well-known creative professionals discussign the future zof digital media 
in th artistic landscape."""

corrected_paragraph_unigram = correct_text(test_paragraph_with_errors)
corrected_paragraph_bigram = correct_text_bigram(test_paragraph_with_errors)
corrected_paragraph_bigram_with_laplase_smoothing = correct_text_bigram_with_laplase_smoothing(test_paragraph_with_errors)

unigram_accuracy = calculate_word_accuracy(test_paragraph, test_paragraph_with_errors, corrected_paragraph_unigram)
bigram_accuracy = calculate_word_accuracy(test_paragraph, test_paragraph_with_errors, corrected_paragraph_bigram)
bigram_accuracy_with_laplase_smoothing = calculate_word_accuracy(test_paragraph, test_paragraph_with_errors, corrected_paragraph_bigram_with_laplase_smoothing)

print(f"Unigram accuracy: {unigram_accuracy}")
print(f"Bigram accuracy: {bigram_accuracy}")
print(f"Bigram with Laplase smoothing accuracy: {bigram_accuracy_with_laplase_smoothing}")

Unigram accuracy: 0.08695652173913043
Bigram accuracy: 0.13043478260869565
Bigram with Laplase smoothing accuracy: 0.11594202898550725


### Test the fragment from the book "The Lord of The Rings" by John Ronald Reuel Tolkien

In [339]:
# text fragment from "The Lord of The Rings"
test_text = "This tale grew in the telling, until it became a history of the Great War of the Ring and included many glimpses of the yet more ancient history that preceded it. It was begun soon after The Hobbit was written and before its publication in 1937; but I did not go on with this sequel, for I wished first to complete and set in order the myth- ology and legends of the Elder Days, which had then been taking shape for some years. I desired to do this for my own satisfaction, and I had little hope that other people would be interested in this work, especially since it was primarily linguistic in inspiration and was begun in order to provide the necessary background of ‘history’ for Elvish tongues."
# test_text_with_errors, added_error_num = add_errors(test_text)
test_text_with_errors = """This tle grew in the telling, unti it becwme a history of the Great War of lthe Ring and included many glimpses of the yet more ancient history thft preceded ti. It was bmgun soon after The Hobbit was written and before its publication in 1937; bt I did not go on with this seqel, fro I wished first to complete annd set in order thd myth- ology and legends of the Elder iDays, whijch had hen been making shape for some yeras. I desired to do ths for my own satisfaction, and I had little hope that other speople would be interested in this work, especially since it was primarily linguistic in inspiration nnd was begun in oredr to provide the necessary background of ‘histojry’ fvr Elvish tongues."""


corrected_unigram = correct_text(test_text_with_errors)
corrected_bigram = correct_text_bigram(test_text_with_errors)
corrected_bigram_with_laplase_smoothing = correct_text_bigram_with_laplase_smoothing(test_text_with_errors)
# correct_with_beam_search = correct_text_bigram_beam_search(test_text_with_errors, 3)
print("Unigram accuracy:", calculate_word_accuracy(test_text, test_text_with_errors, corrected_unigram))
print("Bigram accuracy:", calculate_word_accuracy(test_text, test_text_with_errors, corrected_bigram))
print("Bigram with Laplase smoothing accuracy:", calculate_word_accuracy(test_text, test_text_with_errors, corrected_bigram_with_laplase_smoothing))
# print("Beam search accuracy:", calculate_word_accuracy(test_text, test_text_with_errors, correct_with_beam_search))
    

Unigram accuracy: 0.11504424778761062
Bigram accuracy: 0.13274336283185842
Bigram with Laplase smoothing accuracy: 0.1415929203539823


## Evaluation results

I decided to test the implemented models on 5 test cases.

✏️ **without context** (to test the model ability of correcting single words)

1. Norvig's unit tests
2. Words from file `spell-testset1.txt`

📝 **with context** (to test the model ability to capture the context)

3. Sentence created from the bigrams listed in `count_2w.txt`.
4. Paragraph created from the bigrams listed in `count_2w.txt`.
5. Fragment from the book "The Lord of The Rings".

The errors to the context test cases were added using function `add_errors()` that randomly addes mistakes to the correct words with the given error frequency.

### Results

I calculated the model accuracy as the (number of correctly corrected words)/(the number of words that had errors and were present in the corpus).

| Test case                  | Unigram | Simple bigram | Bigram with Laplase smoothing |
|------------------------------------|---------|---------|---------|
| Norvig's unit tests               | passed  | passed  | passed   |
| Words test file (`spell-testset1.txt` without context) | 74%     | 41%     | 49%     |
| Accuracy on test sentence         | 0.25    | 0.25    | 0.25     |
| Accuracy on the test paragraph    | 0.087    | 0.13    | 0.116   |
| Accuracy on "The Lord of The Rings" fragment | 0.115   | 0.133   | 0.142   |

The original Norwig's solution reached 75% on the words from `spell-testset1.txt` while my implementation achieves 74%. I think that these results are really close and this could happen because as I mentioned before in my case `big.txt` dataset was slightly larger.

From the obtained results we can see that bigram models performed worse than the unigram model on single word test cases. Nevertheless, in the test cases with the context bigram models performed better and the accuracy was higher for the bigram model with Laplase smoothing 😊


## Future work
1. Keyboard layout: correct probabilities of candidates based on their location on the keyboard.
2. Use characters changing frequencies

In the website with Norvig model I found the file `count_1edit.txt` with the mispelling frequency of the given characters.

4. Pay attention to punctuation.

I believe that if we include punctuation into N-grams, the results of the model will improve because some words are more likely to occur at the beginning of a sentence, some at the end, and conjunctions after commas

5. Use beam search

In the current solution, the model always chooses the word with the highest probability, but it is possible that the word with a lower probability is more appropriate at the moment, and further correction of the text will result in a higher final probability.

## Lessons learned
- develop more maintainable code that can be simply reused


#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)